In [1]:
import pandas as pd
import numpy as np

In [2]:
educacion   = pd.read_csv('~/orga-datos/datos_preprocesados/fiuba_1_postulantes_educacion.csv')

In [3]:
genero_edad = pd.read_csv('~/orga-datos/datos_preprocesados/fiuba_2_postulantes_genero_y_edad.csv')

In [8]:
educacion.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [10]:
educacion['nombre'].value_counts()

Universitario        207752
Secundario           199375
Terciario/Técnico     92632
Otro                  43225
Posgrado              16822
Master                 8299
Doctorado               507
Name: nombre, dtype: int64

In [4]:
nivel_educativo = pd.get_dummies(educacion['nombre'])
educacion = pd.merge(nivel_educativo, educacion, left_index=True, right_index=True, how='right')

In [8]:
educacion.head()

,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,idpostulante,nombre,estado
0,0,0,0,1,0,0,0,NdJl,Posgrado,En Curso
1,0,0,0,0,0,0,1,8BkL,Universitario,En Curso
2,0,0,0,0,0,0,1,1d2B,Universitario,En Curso
3,0,0,0,0,0,0,1,NPBx,Universitario,En Curso
4,0,1,0,0,0,0,0,NPBx,Master,En Curso


In [5]:
def set_estado(reg):
    multiplicador = {'En Curso': 1, 'Abandonado': 2, 'Graduado': 3}[reg['estado']]
    reg[reg['nombre']] = reg[reg['nombre']] * multiplicador
    return reg

educacion = educacion.apply(set_estado, axis=1)
educacion = educacion.drop(axis=1, labels=['nombre', 'estado'])
educacion = educacion.groupby('idpostulante').agg(max)

In [6]:
educacion = educacion.reset_index()
educacion.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0z5Dmrd,0,0,0,0,0,0,3
1,0z5JW1r,0,0,3,0,3,3,2
2,0z5VvGv,0,0,0,0,3,0,0
3,0zB01pE,0,0,0,0,0,3,2
4,0zB026d,0,0,0,0,3,3,0


In [7]:
genero_edad = genero_edad[(genero_edad['sexo'] != 'NO_DECLARA') |  (genero_edad['fechanacimiento'].isna() == False)]

In [8]:
genero_edad[genero_edad.duplicated(subset='idpostulante', keep=False)].sort_values(by='idpostulante').head()

,idpostulante,fechanacimiento,sexo
20462,1126WL,1982-04-09,FEM
230353,1126WL,1981-04-09,FEM
220520,12xKWL,1986-02-05,FEM
10809,12xKWL,1982-02-05,FEM
239049,16lwrN,1964-02-09,FEM


In [9]:
genero_edad = genero_edad.drop_duplicates(subset='idpostulante', keep='last')

In [14]:
postulantes = pd.merge(educacion, genero_edad, on='idpostulante', how='outer')

In [15]:
postulantes.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,0z5Dmrd,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1965-10-22,MASC
1,0z5JW1r,0.0,0.0,3.0,0.0,3.0,3.0,2.0,1971-04-09,MASC
2,0z5VvGv,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1987-10-18,MASC
3,0zB01pE,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1989-12-01,MASC
4,0zB026d,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1978-02-23,FEM


In [16]:
print('Educación:', len(educacion))
print('Género y edad:', len(genero_edad))
print('Total:', len(postulantes))

Educación: 366395
Género y edad: 390214
Total: 396480


In [17]:
for columna in ['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario', 'Terciario/Técnico', 'Universitario']:
    postulantes[postulantes[columna].isna()] = 0
    postulantes[columna] = postulantes[columna].astype('int')
    postulantes[columna] = postulantes[columna].map(lambda x: {1: 'En Curso', 3: 'Graduado', 2: 'Abandonado', 0:0}[x])

In [18]:
postulantes.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,0z5Dmrd,0,0,0,0,0,0,Graduado,1965-10-22,MASC
1,0z5JW1r,0,0,Graduado,0,Graduado,Graduado,Abandonado,1971-04-09,MASC
2,0z5VvGv,0,0,0,0,Graduado,0,0,1987-10-18,MASC
3,0zB01pE,0,0,0,0,0,Graduado,Abandonado,1989-12-01,MASC
4,0zB026d,0,0,0,0,Graduado,Graduado,0,1978-02-23,FEM


In [19]:
postulantes.to_csv('~/orga-datos/datos_preprocesados/postulantes.csv', index=False)